# Data Science for Business - Multilayer Perceptron (MLP) on Ames Housing with Scikit-learn

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
plt.style.use('fivethirtyeight')

## Problem description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. With 76 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset challenges you to predict the final price of each home. More: <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>


## Load data

Load training data from CSV file.

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/refs/heads/main/Session_08/ameshousing.csv')

In [4]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ThreeSsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,141,31770,Pave,none,IR1,Lvl,AllPub,Corner,...,0,0,0,none,0,5,2010,WD,Normal,215000
1,20,RH,80,11622,Pave,none,Reg,Lvl,AllPub,Inside,...,0,120,0,MnPrv,0,6,2010,WD,Normal,105000
2,20,RL,81,14267,Pave,none,IR1,Lvl,AllPub,Corner,...,0,0,0,none,12500,6,2010,WD,Normal,172000
3,20,RL,93,11160,Pave,none,Reg,Lvl,AllPub,Corner,...,0,0,0,none,0,4,2010,WD,Normal,244000
4,60,RL,74,13830,Pave,none,IR1,Lvl,AllPub,Inside,...,0,0,0,MnPrv,0,3,2010,WD,Normal,189900


## Prepare data

First, we will remove some columns that are not useful for our task.

In [5]:
data = data.drop(['YrSold', 'MoSold', 'SaleCondition', 'SaleType'], axis=1)

Next, we will split the data into features (*X*) and labels (*y*) and into training (*X_train, y_train*) and test (*X_test, y_test*) sets.

In [6]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Finally, we will do some feature engineering. It is important to use only information from the training set for feature engineering, and the mechanistically repeat these steps on the test set.

Typically, feature engineering depends strongly on the datatype of the variables. Hence, we will first determine which variables are categorical and which are numerical. Subsequentally, we will transform these variables seperately.

In [7]:
categorical_features = X_train.select_dtypes(include='object').columns
numerical_features = X_train.select_dtypes(exclude='object').columns

The categorical variables must be transformed into numerical representations, e.g., by one-hot encdoing them.

In [8]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(X_train[categorical_features])

X_train_cat = enc.transform(X_train[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_cat = pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out(categorical_features))
X_test_cat = pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out(categorical_features))

In [9]:
X_train_cat.head()

,MSZoning_A(agr),MSZoning_C(all),MSZoning_FV,MSZoning_I(all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The numerical variables will be standardized, that is, we will subtract the mean and divide by the standard deviation. This is especially important for LASSO, as all coefficients need to be comparable in terms of units and magnitudes.

In [10]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features]) 

X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

X_train_num = pd.DataFrame(X_train_num, columns=numerical_features)
X_test_num = pd.DataFrame(X_test_num, columns=numerical_features)

In [11]:
X_train_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,MiscVal
0,-0.871817,0.667826,0.033810,0.673941,-0.526415,0.181084,-0.381277,0.531409,-0.978369,-0.293998,...,0.614268,0.339813,0.047615,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
1,0.062906,-1.717704,2.307082,-0.766750,-0.526415,-0.115603,-0.814347,-0.569155,-0.427633,4.191480,...,-0.917482,0.339813,0.325180,3.139485,-0.695967,-0.369038,-0.098812,3.744734,-0.067407,-0.09315
2,0.763949,0.369635,-0.035514,-1.487095,-0.526415,-0.280430,-1.054941,-0.569155,-0.978369,-0.293998,...,-0.917482,0.339813,-0.032361,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
3,0.763949,0.071444,-0.363746,-1.487095,-0.526415,-0.708978,-1.632368,-0.569155,-0.978369,-0.293998,...,-0.917482,0.339813,-0.229950,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
4,3.100756,0.160901,-0.310697,-1.487095,0.378216,-1.664971,-1.632368,-0.569155,-0.978369,-0.293998,...,-0.917482,-2.337571,-2.205835,-0.753959,-0.695967,1.839369,-0.098812,-0.286876,-0.067407,-0.09315


Let's fuse the enginnered categorical and numerical variables again.

In [12]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [13]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,-0.871817,0.667826,0.033810,0.673941,-0.526415,0.181084,-0.381277,0.531409,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.062906,-1.717704,2.307082,-0.766750,-0.526415,-0.115603,-0.814347,-0.569155,-0.427633,4.191480,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.763949,0.369635,-0.035514,-1.487095,-0.526415,-0.280430,-1.054941,-0.569155,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.763949,0.071444,-0.363746,-1.487095,-0.526415,-0.708978,-1.632368,-0.569155,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3.100756,0.160901,-0.310697,-1.487095,0.378216,-1.664971,-1.632368,-0.569155,-0.978369,-0.293998,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Neural Network

In the following, we will create a neural network with one hidden layer comprising 256 neurons. We will use the ReLU activation function and train for 1000 epochs.

In [14]:
model_mlp = MLPRegressor(hidden_layer_sizes=(256,), activation = 'relu', learning_rate_init = 0.001, max_iter=1000, solver='adam', random_state=42)

In [15]:
model_mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(256,), max_iter=1000, random_state=42)

In [16]:
# Training data
pred_train = model_mlp.predict(X_train)
r2_train = r2_score(y_train, pred_train)
rmse_train = mean_squared_error(y_train, pred_train, squared=False)
print('R2 on training set:', round(r2_train, 2))
print('RMSE on training set:', round(rmse_train, 2))

print("===")

# Test data
pred_test = model_mlp.predict(X_test)
r2_test = r2_score(y_test, pred_test)
rmse_test = mean_squared_error(y_test, pred_test, squared=False)
print('R2 on test set:', round(r2_test, 2))
print('RMSE on test set:', round(rmse_test, 2))

R2 on training set: 0.86
RMSE on training set: 28703.34
===
R2 on test set: 0.87
RMSE on test set: 31892.31
